In [27]:
import pandas as pd
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))


In [12]:
#Reading the csv files
cities = pd.read_csv('../data/Cities/Chart data.csv', index_col=False)
cities.name = 'cities'
Content_type = pd.read_csv('../data/Content type/Chart data.csv', index_col=False)
Content_type.name = 'Content_type'
Device_type = pd.read_csv('../data/Device type/Chart data.csv', index_col=False)
Device_type.name = 'Device_type'
Geography = pd.read_csv('../data/Geography/Chart data.csv', index_col=False)
Geography.name = 'Geography'
New_and_returning_viewers = pd.read_csv('../data/New and returning viewers/Chart data.csv', index_col=False)
New_and_returning_viewers.name = 'New_and_returning_viewers'
Operating_system = pd.read_csv('../data/Operating system/Chart data.csv', index_col=False)
Operating_system.name = 'Operating_system'
Sharing_service = pd.read_csv('../data/Sharing service/Chart data.csv', index_col=False)
Sharing_service.name = 'Sharing_service'
Subscription_source = pd.read_csv('../data/Subscription source/Chart data.csv', index_col=False)
Subscription_source.name = 'Subscription_source'
Subscription_status = pd.read_csv('../data/Subscription status/Chart data.csv', index_col=False)
Subscription_status.name = 'Subscription_status'
Subtitles_and_CC = pd.read_csv('../data/Subtitles and CC/Chart data.csv', index_col=False)
Subtitles_and_CC.name = 'Subtitles_and_CC'
Traffic_source = pd.read_csv('../data/Traffic source/Chart data.csv', index_col=False)
Traffic_source.name = 'Traffic_source'

In [13]:
cities.head()

,Date,Cities,City name,Views
0,2020-06-28,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0
1,2020-06-29,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0
2,2020-06-30,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0
3,2020-07-01,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0
4,2020-07-02,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0


In [14]:
Traffic_source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11511 entries, 0 to 11510
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            11511 non-null  object
 1   Traffic source  11511 non-null  object
 2   Views           11511 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 269.9+ KB


In [15]:
Traffic_source_table = pd.read_csv('../data/Traffic source/Table data.csv', index_col=False)
Traffic_source_table.name = 'Traffic_table'

In [17]:
Traffic_source_table.head()

,Traffic source,Views,Watch time (hours),Average view duration,Impressions,Impressions click-through rate (%)
0,Total,26625.0,2157.3497,0:04:51,606516.0,2.45
1,Channel pages,8444.0,780.2340,0:05:32,218825.0,2.95
2,Browse features,4759.0,321.8387,0:04:03,109132.0,3.43
3,Direct or unknown,4123.0,327.4812,0:04:45,NaN,NaN
4,YouTube search,3123.0,232.8650,0:04:28,93094.0,2.73


In [18]:
Content_type.head()

,Date,Content type,Views
0,2020-06-28,Other,0
1,2020-06-29,Other,0
2,2020-06-30,Other,0
3,2020-07-01,Other,0
4,2020-07-02,Other,0


In [20]:
Content_type['Content type'].nunique()

2